## 04.05.2020

The possibility to run this notebook has been brought forward.

This adds some prerequisites and adaptations to cope with what is not available yet.

Note:
- Mappings have not been reviewed yet ([DKE-143](https://bbpteam.epfl.ch/project/issues/browse/DKE-143))
- **The configuration `bbp-staging-forge.yml` should be completed with the values available at the time of running.**

Prerequisites:
- The Python package `kgforge` is installed.
- This notebook is running from the subdirectory `/examples/notebooks/use-cases/` of the cloned package repository.

Adaptations:
- In a cell, the commented line is the targeted behaviour which is not available yet. A replacement is provided meanwhile.

---

## DISCLAMER

This notebook might not be executable as-is at the moment you try to do so (missing data, access, implementations, ...).

The goal here is to demonstrate what can be done with the framework in specific use cases.

---

## Content

This notebook focuses on demonstrating an end-to-end workflow for the Blue Brain Knowledge Graph.

- Tools installation

- Session configuration

- A - Data Integration
  - Retrieve human neuron morphologies from the Allen Cell Types Database
  - Load the complete metadata of the neuron morphologies from Allen
  - Consult the list of managed data sources and mappings to the Blue Brain Knowledge Graph
  - Map the neuron morphologies from Allen to the Blue Brain Knowledge Graph
  - Verify the created entities from Allen conform to the Blue Brain Knowledge Graph Schema
  - Add the created entities from Allen to the Blue Brain Knowledge Graph


- B - Data Exploration
  - Discover which neuron morphologies are already in the Blue Brain Knowledge Graph
  - Select neuron morphologies in the cortical layer V and with intact apical dendrites
  - Regroup as a dataset a selection of neuron morphologies from the Blue Brain Knowledge Graph
  - Give the first revision of the dataset a human-friendly name


- C - Data Analytics
  - Retrieve a specific dataset at a given version from the Blue Brain Knowledge Graph
  - Download the reconstruction files of the neuron morphologies of the dataset
  - Perform a topological analysis of the neuron morphologies from the dataset
  - Register the analysis result with its provenance into the Blue Brain Knwoledge Graph

---

## Tools installation

In [ ]:
# ! pip install kgforge allensdk tmd
! pip install allensdk tmd

---

## Session configuration

In [ ]:
import getpass

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
from kgforge.specializations.resources import Dataset

Please enter your BBP token:

In [ ]:
token = getpass.getpass()

In [ ]:
# forge = KnowledgeGraphForge("bbp-prod-forge.yml", bucket="bbp/<project>", token=token)
forge = KnowledgeGraphForge("bbp-staging-forge.yml", token=token)

---

## A - Data Integration

### Retrieve human neuron morphologies from the Allen Cell Types Database

In [ ]:
from allensdk.core.cell_types_cache import CellTypesCache

In [ ]:
from allensdk.api.queries.cell_types_api import CellTypesApi

In [ ]:
ALLEN_DIR = "allen_cell_types_database"

In [ ]:
ctc = CellTypesCache(manifest_file=f"{ALLEN_DIR}/manifest.json")

In [ ]:
human_cells = ctc.get_cells(species=[CellTypesApi.HUMAN], require_reconstruction=True)

In [ ]:
CELLS_LIMIT = 2

In [ ]:
human_cell_ids = [x["id"] for x in human_cells][:CELLS_LIMIT]

In [ ]:
human_cell_reconstructions = [ctc.get_reconstruction(x) for x in human_cell_ids]

### Load the complete metadata of the neuron morphologies from Allen

In [ ]:
import json

In [ ]:
with open(f"{ALLEN_DIR}/cells.json") as f:
    allen_cell_types_metadata = json.load(f)

In [ ]:
human_cell_metadata = [x for x in allen_cell_types_metadata if x["specimen__id"] in human_cell_ids]

### Consult the list of managed data sources and mappings to the Blue Brain Knowledge Graph

In [ ]:
forge.sources()

In [ ]:
DATA_SOURCE = "allen-cell-types-database"

In [ ]:
forge.mappings(DATA_SOURCE)

### Map the neuron morphologies from Allen to the Blue Brain Knowledge Graph

In [ ]:
subject_mapping = forge.mapping("Subject", DATA_SOURCE)

In [ ]:
print(subject_mapping)

In [ ]:
patchedcell_mapping = forge.mapping("PatchedCell", DATA_SOURCE)

In [ ]:
print(patchedcell_mapping)

In [ ]:
neuronmorphology_mapping = forge.mapping("NeuronMorphology", DATA_SOURCE)

In [ ]:
print(neuronmorphology_mapping)

In [ ]:
mappings = [subject_mapping, patchedcell_mapping, neuronmorphology_mapping]

In [ ]:
resources = forge.map(human_cell_metadata, mappings)

### Verify the created entities from Allen conform to the Blue Brain Knowledge Graph Schema

In [ ]:
forge.validate(resources)

### Add the created entities from Allen to the Blue Brain Knowledge Graph

In [ ]:
forge.register(resources)

---

## B - Data Exploration

In [ ]:
# p = forge.paths("NeuronMorphology")
p = forge.paths("ReconstructedPatchedCell")

### Discover which neuron morphologies are already in the Blue Brain Knowledge Graph

In [ ]:
# results = forge.search(p.type == "NeuronMorphology")
results = forge.search(p.type == "ReconstructedPatchedCell")

In [ ]:
len(results)

In [ ]:
DISPLAY_LIMIT = 25

In [ ]:
forge.as_dataframe(results[:DISPLAY_LIMIT])

### Select neuron morphologies in the cortical layer V and with intact apical dendrites

In [ ]:
neuronmorphologies = forge.search(p.type == "NeuronMorphology",
                                  p.brainLocation.layer.label == "5",
                                  p.apicalDendrite == "intact")

In [ ]:
len(neuronmorphologies)

In [ ]:
forge.as_dataframe(neuronmorphologies[:DISPLAY_LIMIT])

### Regroup as a dataset a selection of neuron morphologies from the Blue Brain Knowledge Graph

In [ ]:
from uuid import uuid4

In [ ]:
DATASET_ID = forge.format("identifier", "datasets", str(uuid4()))

In [ ]:
# FIXME According to Neuroshapes on 09.08.2019, 'subject', 'brainLocation' are also required.
# FIXME Property 'hasPart' is not part of the Dataset shape at the moment (09.08.2019).
dataset = Dataset(forge,
                  id=DATASET_ID,
                  type=["Dataset", "NeuronMorphology"],
                  name="All layer 5 morphologies with intact apical dendrites",
                  description="Neuron morphologies to be used for Topological Morphology Descriptor analysis")

In [ ]:
CONTRIBUTOR_NAME = "Jane Doe"

In [ ]:
agent = forge.resolve(CONTRIBUTOR_NAME, scope="entities", target="agents", type="Person")

In [ ]:
AGENT_ID = agent.id

In [ ]:
dataset.add_contribution(AGENT_ID)

In [ ]:
dataset.add_parts(neuronmorphologies)

In [ ]:
forge.register(dataset)

### Give the first revision of the dataset a human-friendly name

In [ ]:
VERSION_NAME = "v2019-08-20"

In [ ]:
forge.tag(dataset, VERSION_NAME)

---

## C - Data Analytics

In [ ]:
import tmd

In [ ]:
from tmd.view import plot

### Retrieve a specific dataset at a given version from the Blue Brain Knowledge Graph

In [ ]:
dataset = forge.retrieve(id=DATASET_ID, version=VERSION_NAME)

### Download the reconstruction files of the neuron morphologies of the dataset

In [ ]:
DOWNLOAD_DIR = f"./reconstructions_{VERSION_NAME}/"

In [ ]:
dataset.download("parts", DOWNLOAD_DIR)

### Perform a topological analysis of the neuron morphologies from the dataset

In [ ]:
pop = tmd.io.load_population(DOWNLOAD_DIR)

In [ ]:
phs = [tmd.methods.get_persistence_diagram(x.apical[0]) for x in pop.neurons]

In [ ]:
phs_flattened = tmd.analysis.collapse(phs)

#### Visualize the persistence diagram

In [ ]:
plot.diagram(phs_flattened)

#### Visualize the persistence barcode

In [ ]:
plot.barcode(phs_flattened)

#### Visualize and save the persistence image

In [ ]:
ANALYSIS_DIR = "analysis"

In [ ]:
ANALYSIS_FILENAME = "persistence_image"

In [ ]:
plot.persistence_image(phs_flattened, output_path=ANALYSIS_DIR, output_name=ANALYSIS_FILENAME)

### Register the analysis result with its provenance into the Blue Brain Knwoledge Graph

In [ ]:
# FIXME According to Neuroshapes on 09.08.2019, 'used' (2 times) and 'generated' are also required.
analysis = Dataset(forge, type=["Dataset", "Analysis"], name="Persistence image")

In [ ]:
analysis.add_contribution(AGENT_ID)

In [ ]:
analysis.add_derivation(dataset)

In [ ]:
analysis.add_distribution(f"./{ANALYSIS_DIR}/{ANALYSIS_FILENAME}")

In [ ]:
forge.register(analysis)